In [1]:
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import interact, widgets ,  IntRangeSlider , IntSlider , FloatSlider
import numpy as np
from scipy.stats import gaussian_kde

In [2]:
df = pd.read_csv(r'C:\Users\devin\Desktop\ASSET ARBITRAGE\final_dataset_preprocessed.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6191 entries, 0 to 6190
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            6191 non-null   object 
 1   Close           6191 non-null   float64
 2   High            6191 non-null   float64
 3   Low             6191 non-null   float64
 4   Open            6191 non-null   float64
 5   Volume          6191 non-null   float64
 6   Price           6191 non-null   float64
 7   Returns         6191 non-null   float64
 8   Price_scaled    6191 non-null   float64
 9   Volume_scaled   6191 non-null   float64
 10  Returns_scaled  6191 non-null   float64
dtypes: float64(10), object(1)
memory usage: 532.2+ KB


In [3]:
df.describe()

,Close,High,Low,Open,Volume,Price,Returns,Price_scaled,Volume_scaled,Returns_scaled
count,6191.000000,6191.000000,6191.000000,6191.000000,6.191000e+03,6191.000000,6191.000000,6.191000e+03,6.191000e+03,6.191000e+03
mean,1362.245404,1369.522903,1354.601257,1362.148236,1.483006e+04,1362.955144,0.001008,-1.101795e-16,2.754486e-17,2.295405e-17
std,605.920265,609.191935,601.995322,605.395010,5.595973e+04,608.975675,0.019006,1.000081e+00,1.000081e+00,1.000081e+00
min,374.799988,375.799988,375.799988,375.799988,0.000000e+00,374.799988,-0.032866,-1.622782e+00,-2.650345e-01,-1.782410e+00
25%,956.799988,962.145006,951.265012,956.299988,4.300000e+01,957.750000,-0.007456,-6.654418e-01,-2.642661e-01,-4.453611e-01
50%,1291.490000,1295.900024,1285.699951,1291.599976,1.890000e+02,1290.800049,0.000000,-1.184956e-01,-2.616568e-01,-5.302549e-02
75%,1729.549988,1739.400024,1721.350036,1730.599976,9.060000e+02,1728.850036,0.009484,6.008852e-01,-2.488430e-01,4.460051e-01
max,3820.899902,3827.600098,3775.300049,3775.300049,1.236670e+06,3820.899902,0.034894,4.036521e+00,2.183603e+01,1.783054e+00


In [4]:
# Make sure Date is datetime
df['Date'] = pd.to_datetime(df['Date'])

## Insight 1 — Overall Gold Price Trend
Line plot of price over time and percentage growth from first to last observation.

In [5]:
# Function to plot (without mask)
def plot_gold_price(start_date, end_date):
    # It selects the data within the range given in dataframe
    filtered_df = df.loc[(df['Date'] >= pd.to_datetime(start_date)) & (df['Date'] <= pd.to_datetime(end_date))]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=filtered_df['Date'],y=filtered_df['Close'],mode='lines+markers',name='Gold Price',line=dict(color='blue')))
    fig.update_layout(title='Gold Price Trend Over Time',xaxis_title='Date',yaxis_title='Price',template='plotly_white')
    # Add growth annotation if data exists (Means it adds a arrow or a kind off label at the end to hows stats)
    if not filtered_df.empty:
        initial = filtered_df['Close'].iloc[0]
        final = filtered_df['Close'].iloc[-1]
        growth = ((final - initial) / initial) * 100
        fig.add_annotation(x=filtered_df['Date'].iloc[-1],y=filtered_df['Close'].iloc[-1],text=f"Initial: {initial:.2f}<br>Final: {final:.2f}<br>Growth: {growth:.2f}%",showarrow=True,arrowhead=1,ax=-50,ay=-50)
    fig.show()
# Widgets for start and end date
start_widget = widgets.DatePicker(description='Start Date',value=df['Date'].min(),)
end_widget = widgets.DatePicker(description='End Date',value=df['Date'].max(),)
# Interactive dashboard
interact(plot_gold_price, start_date=start_widget, end_date=end_widget)

interactive(children=(DatePicker(value=Timestamp('2004-01-03 00:00:00'), description='Start Date', step=1), Da…

<function __main__.plot_gold_price(start_date, end_date)>

## Insight 2 — Yearly Average Price Trend
Compute average price per year and plot to identify peak years.

In [6]:
# Assuming df is already defined and has 'Date' and 'Price' columns
df['Year'] = df['Date'].dt.year
# groupby("year") : this group price of a year , .mean() this calculate average of price per year , reset_index : this reset the index mean clean dataframe after calculating for particular year
yearly_avg = df.groupby('Year')['Price'].mean().reset_index()
# Function to update interactive chart
def update_chart(year_range):
    start, end = year_range
    filtered = yearly_avg[(yearly_avg['Year'] >= start) & (yearly_avg['Year'] <= end)]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=filtered['Year'],y=filtered['Price'],mode='lines+markers',line=dict(width=2),marker=dict(size=8),name='Avg Gold Price'))
    fig.update_layout(title=f'Average Gold Price Per Year ({start}-{end})',xaxis_title='Year',yaxis_title='Average Price',template='plotly_white',hovermode='x unified', width=900,height=450)
    fig.show()
# Interactive slider
year_slider = IntRangeSlider(value=[yearly_avg['Year'].min(), yearly_avg['Year'].max()],min=yearly_avg['Year'].min(),max=yearly_avg['Year'].max(),step=1,description='Year Range:',continuous_update=True)
# Link slider to chart
interact(update_chart, year_range=year_slider)
# Print top performing year
top_year = yearly_avg.loc[yearly_avg['Price'].idxmax(), 'Year']
top_price = yearly_avg['Price'].max()
print(f"Year with highest average price: {top_year} (avg = {top_price:.2f})")

interactive(children=(IntRangeSlider(value=(2004, 2025), description='Year Range:', max=2025, min=2004), Outpu…

Year with highest average price: 2025 (avg = 3210.69)


## Insight 3 - return distribution
makes us understand how daily returns are distributed over time

In [7]:
# Drop NaNs bcuz that will cause zero bin error
df = df.dropna(subset=["Returns"])
# Compute mean
mean_val = df["Returns"].mean()
# Function to update histogram interactively
def update_histogram(bins):
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=df["Returns"],nbinsx=bins, marker_color='blue',opacity=0.7,name='Returns Distribution'))

    # Density curve
    density = gaussian_kde(df["Returns"])
    x_vals = np.linspace(df["Returns"].min(), df["Returns"].max(), 200)
    fig.add_trace(go.Scatter(x=x_vals,y=density(x_vals)*len(df["Returns"])*np.diff(np.histogram(df["Returns"], bins=bins)[1])[0],mode='lines',line=dict(color='purple', width=2),name='Density'))
    fig.add_vline(x=mean_val,line=dict(color='red', dash='dash'),annotation_text=f"Mean: {mean_val:.4f}",annotation_position="top left")
    fig.update_layout(title=f"Distribution of Daily Returns (Bins: {bins})",xaxis_title="Daily Returns",yaxis_title="Frequency",template="plotly_white",width=900,height=450,legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))
    fig.show()

# Interactive slider
interact(update_histogram,bins=IntSlider(value=50, min=10, max=100, step=5, description="Bins:"))

interactive(children=(IntSlider(value=50, description='Bins:', min=10, step=5), Output()), _dom_classes=('widg…

<function __main__.update_histogram(bins)>

## Insight 4 — Daily Volatility
Plot daily percent-change to inspect volatility and compute average daily volatility.

In [8]:
# Calculate Daily % Change (Returns)
#pct_change means (today's price - yesterday's price / yesterday's price)
#here daily % change is same as returns 
df['Returns'] = df['Price'].pct_change() * 100
# Drop first NaN (as it can cause errors)
df = df.dropna(subset=['Returns'])
df['Year'] = df['Date'].dt.year
def update_volatility_chart(year_range):
    start, end = year_range
    filtered = df[(df['Year'] >= start) & (df['Year'] <= end)]
    vol = filtered['Returns'].std()
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=filtered['Date'],y=filtered['Returns'],mode='lines',line=dict(width=1.8),name='Returns'))
    fig.update_layout(title=f"📈 Daily Gold Price Change (%) — {start}-{end}<br><sup>Volatility (Std Dev): {vol:.2f}%</sup>",xaxis_title="Date",yaxis_title="Daily % Change",template="plotly_white",hovermode="x unified",width=950,height=450)
    fig.show()
year_slider = IntRangeSlider(value=[df['Year'].min(), df['Year'].max()],min=df['Year'].min(),max=df['Year'].max(),step=1,description='Year Range:',continuous_update=True)
interact(update_volatility_chart, year_range=year_slider)
# Print overall volatility
overall_vol = df['Returns'].std()
print(f"Average daily volatility (std dev of daily % change): {overall_vol:.2f}%")

interactive(children=(IntRangeSlider(value=(2004, 2025), description='Year Range:', max=2025, min=2004), Outpu…

Average daily volatility (std dev of daily % change): 4.56%


## Insight 5 — Rolling Mean (Smoothed Trend)
Plot the 30-day rolling mean to visualize the smoothed trend and detect reversals.

In [9]:
def update_rolling_chart(window):
    df['Rolling Mean'] = df['Price'].rolling(window=window).mean()
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['Date'],y=df['Price'],mode='lines',line=dict(color='lightgray', width=1.5),name='Actual Price'))
    fig.add_trace(go.Scatter(x=df['Date'],y=df['Rolling Mean'],mode='lines',line=dict(color='blue', width=2.5),name=f'{window}-Day Rolling Mean'))
    fig.update_layout(title=f"📊 Gold Price Trend with {window}-Day Moving Average",xaxis_title="Date",yaxis_title="Price",template="plotly_white",hovermode="x unified",width=950,height=500,legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))
    fig.show()

interact(update_rolling_chart,window=IntSlider(value=30, min=5, max=90, step=5, description="Rolling Window:"))
print("The rolling mean removes short-term noise and highlights long-term price trends.\n"
      " When the price crosses above/below the moving average, it can signal potential buy/sell opportunities.")

interactive(children=(IntSlider(value=30, description='Rolling Window:', max=90, min=5, step=5), Output()), _d…

The rolling mean removes short-term noise and highlights long-term price trends.
 When the price crosses above/below the moving average, it can signal potential buy/sell opportunities.


## Insight 6 — Price Spikes Detection
Detect and plot days with large daily changes (e.g., >2%). These are potential arbitrage or shock events.

In [10]:
# Ensure the data is precomputed
df = df.dropna(subset=['Returns'])
# Interactive function
def update_spike_chart(threshold):
    spikes = df[df['Returns'].abs() > threshold]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['Date'],y=df['Price'],mode='lines',line=dict(width=1.5, color='lightgray'),name='Gold Price'))
    fig.add_trace(go.Scatter(x=spikes['Date'],y=spikes['Price'],mode='markers',marker=dict(color='red', size=8, symbol='circle'),name=f'Spikes (> {threshold:.1f}%)',hovertemplate='Date: %{x}<br>Price: %{y:.2f}<br>Change: %{customdata:.2f}%',customdata=spikes['Returns']))
    fig.update_layout(title=f"📈 Gold Price Spikes (> {threshold:.1f}% Returns)",xaxis_title="Date",yaxis_title="Price",template="plotly_white",width=950,height=450,legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))
    fig.show()
    print(f"Number of spike days (> {threshold:.1f}% change): {len(spikes)}")
    display(spikes.sort_values('Returns', key=abs, ascending=False).head(10))

interact(update_spike_chart,threshold=FloatSlider(value=2.0, min=0.5, max=10.0, step=0.5, description='Threshold (%):'))

interactive(children=(FloatSlider(value=2.0, description='Threshold (%):', max=10.0, min=0.5, step=0.5), Outpu…

<function __main__.update_spike_chart(threshold)>

## Insight 7 - Top Profitable Days (Biggest Positive Moves)
Which days gold gave the highest daily % gain.

In [11]:
df = df.dropna(subset=['Returns'])
def update_top_gains(n):
    top_gains = df.sort_values("Returns", ascending=False).head(n)
    fig = go.Figure()
    fig.add_trace(go.Bar(x=top_gains["Date"],y=top_gains["Returns"],marker_color='blue',hovertemplate=("Date: %{x}<br>""Price: %{customdata[0]:.2f}<br>""Change: %{y:.2f}%"),customdata=top_gains[["Price"]].values,name='Top Gain Days'))
    fig.update_layout(title=f"Top {n} Gold Gain Days (Best Days to Sell)",xaxis_title="Date",yaxis_title="Returns",template="plotly_white",width=950,height=450,xaxis_tickangle=-45)
    fig.show()
    print(f"These are the top {n} days gold surged the most (good SELL days):")
    display(top_gains[["Date", "Price", "Returns"]])

interact(update_top_gains,n=IntSlider(value=10, min=5, max=30, step=1, description="Top N Days:"))

interactive(children=(IntSlider(value=10, description='Top N Days:', max=30, min=5), Output()), _dom_classes=(…

<function __main__.update_top_gains(n)>

## Insight 8 - Worst Days (Biggest Negative Moves)
Which days gold dropped sharply → best buying opportunities.

In [12]:
df = df.dropna(subset=['Returns'])
def update_top_losses(n):
    # Get top N loss days (most negative % change)
    top_losses = df.sort_values("Returns", ascending=True).head(n)
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=top_losses["Date"],y=top_losses["Returns"],marker_color='crimson',customdata=top_losses[["Price"]].values,hovertemplate=
            "Date: %{x}<br>" +
            "Price: %{customdata[0]:.2f}<br>" +
            "Change: %{y:.2f}%" ,name='Top Loss Days'))
    fig.update_layout(title=f"Top {n} Gold Loss Days (Best Days to Buy)",xaxis_title="Date",yaxis_title="Returns",template="plotly_white",width=950,height=450,xaxis_tickangle=-45)
    fig.show()
    print(f"These are the top {n} days gold dropped the most (good BUY days):")
    display(top_losses[["Date", "Price", "Returns"]])
interact(update_top_losses,n=IntSlider(value=10, min=5, max=30, step=1, description="Top N Days:"))

interactive(children=(IntSlider(value=10, description='Top N Days:', max=30, min=5), Output()), _dom_classes=(…

<function __main__.update_top_losses(n)>

## Insight 9 - Best Months to Trade
Which months (across all years) usually give higher average returns.

In [13]:
df = df.dropna(subset=['Returns'])
df['Month'] = df['Date'].dt.month
monthly_avg = df.groupby("Month")["Returns"].mean().reset_index()
def update_monthly_chart(threshold):
    colors = ['blue' if x <= threshold else 'orange' for x in monthly_avg['Returns']]
    fig = go.Figure()
    fig.add_trace(go.Bar(x=monthly_avg["Month"],y=monthly_avg["Returns"],marker_color=colors,text=monthly_avg["Returns"].round(2),textposition='auto',name='Monthly Avg % Change'))
    fig.update_layout(title=f"📊 Average Monthly Return (%) — Highlight > {threshold:.2f}%",xaxis_title="Month",yaxis_title="Average % Change",template="plotly_white",width=900,height=450)
    fig.show()
    print("Months with higher avg % change are more profitable trading periods:")
    display(monthly_avg)
interact(update_monthly_chart,threshold=FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.1, description="Highlight > %:"))

interactive(children=(FloatSlider(value=0.0, description='Highlight > %:', max=1.0, min=-1.0), Output()), _dom…

<function __main__.update_monthly_chart(threshold)>

## Insight 10 - Weekday Analysis
Which weekdays (Mon–Fri) are most volatile/profitable.

In [14]:
df = df.dropna(subset=['Returns'])
df['Weekday'] = df['Date'].dt.day_name()
weekday_avg = df.groupby("Weekday")["Returns"].mean().reindex(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]).reset_index()
def update_weekday_chart(threshold):
    colors = ['blue' if x <= threshold else 'orange' for x in weekday_avg['Returns']]
    fig = go.Figure()
    fig.add_trace(go.Bar(x=weekday_avg["Weekday"],y=weekday_avg["Returns"],marker_color=colors,text=weekday_avg["Returns"].round(2),textposition='auto',name='Avg % Change'))
    fig.update_layout(
        title=f"Average Return by Weekday — Highlight > {threshold:.2f}%",xaxis_title="Weekday",yaxis_title="Average % Change",template="plotly_white",width=850,height=450)
    fig.show()
    print("Certain weekdays show more movement → better for short-term trades.")
    display(weekday_avg)
interact(update_weekday_chart,threshold=FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.05, description="Highlight > %:"))

interactive(children=(FloatSlider(value=0.0, description='Highlight > %:', max=1.0, min=-1.0, step=0.05), Outp…

<function __main__.update_weekday_chart(threshold)>

## Insight 11 - Most Volatile Periods (Rolling Volatility)
Detect windows of high risk/high opportunity.

In [15]:
df = df.dropna(subset=['Returns'])
def update_rolling_volatility(window):
    df['Rolling_Volatility'] = df['Returns'].rolling(window=window).std()
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['Date'],y=df['Rolling_Volatility'],mode='lines',line=dict(color='blue', width=2),name=f'{window}-Day Rolling Volatility'))
    fig.update_layout(title=f" {window}-Day Rolling Volatility (%)",xaxis_title="Date",yaxis_title="Volatility (%)",template="plotly_white",width=950,height=450,hovermode="x unified")
    fig.show()
    print("Periods with higher volatility are ideal for arbitrage opportunities.")
interact(update_rolling_volatility, window=IntSlider(value=30, min=10, max=60, step=5, description="Rolling Window:"))

interactive(children=(IntSlider(value=30, description='Rolling Window:', max=60, min=10, step=5), Output()), _…

<function __main__.update_rolling_volatility(window)>

## Summary: Actionable Trading Insights
Best SELL Days: Top surge days (price spiked)

Best BUY Days: Top drop days (price crashed)

Best Months: Seasonality of returns

Best Weekdays: Which weekdays are most profitable historically

High-Volatility Periods: Detect risky but lucrative arbitrage windows